# A 5-minute tour of SynapseML

In [0]:
from pyspark.sql import SparkSession
from synapse.ml.core.platform import *

spark = SparkSession.builder.getOrCreate()

if running_on_synapse():
  from synapse.ml.core.platform import materializing_display as display

# Step 1: Load our Dataset

In [0]:
train, test = (
    spark.read.parquet(
        "wasbs://publicwasb@mmlspark.blob.core.windows.net/BookReviewsFromAmazon10K.parquet"
    )
    .limit(1000)
    .cache()
    .randomSplit([0.8, 0.2])
)

display(train.limit(1))

rating,text
1,A Bore Each story has a different author...however they all use the same verbage. Their either all friends or the same person. I do not like this book nor does my husband


# Step 2: Make our Model

In [0]:
from pyspark.ml import Pipeline
from synapse.ml.featurize.text import TextFeaturizer
from synapse.ml.lightgbm import LightGBMRegressor

model = Pipeline(
    stages=[
        TextFeaturizer(inputCol="text", outputCol="features"),
        LightGBMRegressor(featuresCol="features", labelCol="rating"),
    ]
).fit(train)

# Step 3: Predict!

In [0]:
display(model.transform(test).limit(1))

rating text features prediction 1 A Review from the author of WATCH OUT The literature of September 11 is never attacked. When a book speaks of September 11 (or of terrorism in general), it is more or less guaranteed immunity from criticism; it will almost inescapably be greeted with sympathy. Jonathan Safran Foer's EXTREMELY LOUD AND INCREDIBLY CLOSE banks on such sympathy, on such reverence. The narrative, which concerns a nine-year-old boy named Oskar Schell in search of a key that would unlock / unshell the enigma of his dead father (a narrative stolen, in its basic outline, from Guenther Grass's DIE BLECHTROMMEL), could have been written entirely without its scattered references to the terrorist interventions. Nor is this trauma the only one presented in the novel: the others include Hiroshima / Nagasaki, the Staten Island Ferry crash, and the Dresden bombings. Each disaster is generalized to the point at which what is addressed is not a traumatizing event in its specificity, but historical trauma" itself and the overcoming of trauma through bereavement-inspired creation. Oskar, the insufferable brat, attempts to complete the work of mourning for his father, Thomas Schell, Jr., a victim of September 11, by compiling an almanac of self-inflicted wounds, the collage of images and letters which is the book we are "reading"---an almanac which, most likely, is written sometime in the indefinite future (Thomas Shell, Sr.'s manuscript of 4/12/78 is heavily edited (pp. 208-216). Who has done the editing? Almost certainly an older version of his grandson Oskar). If the term, "reading" even applies. Whenever a "pregnant" image is described, Foer literally re-presents it in the form of a pictorial image. When a flock of birds rises into the sky, it is not enough that we read of these birds---we must see them as well. Words may not be left in their invisibility; we are presented with supplementary photographs, illustrations, since mere verbality is not enough. (Indeed, the entire novel oozes with misologos --- the mistrust or hatred of language --- in terms of both its content and its form.) Photographs, yes, and also a superabundance of blank pages and nearly blank pages. Space is not used in the manner it is in the works of Edmond Jabes, for instance. Typography does not substitute for a well-wrought sentence. Foer abrogates himself of all responsibilities --- most specifically, the responsibility to write well. Why bother when the pyrotechnics of typography are at his disposal? As far as the writing is concerned, it is composed of nothing other than mind-numbingly, soul-deadeningly repetitive phrases ("heavy boots," "raison d'etre," etc.) and Sunday school platitudes: "Sometimes one simply wants to disappear" (184); "There's nothing wrong with not understanding yourself" (184); "Everything that's born has to die, which means our lives are like skyscrapers" (245); "How can you say I love you to someone you love?... It's always necessary" (314). Whenever the author writes something that he finds "beautiful" and "true" (165), he congratulates himself on his brilliance and tells us that that thing is "beautiful" and "true." The entire book reeks of such unearned profundity. We also learn that most dust is made up of human detritus---a very deep truth indeed, one that Foer also communicates in his essay, "Emptiness" (originally published in PLAYBOY) with all of the sanctimoniousness and self-righteousness of the faux naif who serves as the center of the novel, a Sunday school lecture in which we learn that famous musicians (Ringo Star) and scientists (Stephen Hawking) are unthreateningly approachable: everything is familiarized. Perhaps it is wrong to criticize Foer for including so many blank pages in EXTREMELY LOUD AND INCREDIBLY CLOSE, since the entire book is a vacuum: null space into which readers may project their own meanings. Joseph Suglia, Ph.D., the author of WATCH OUT" Map(vectorType -> sparse, length -> 262144, indices -> L

# Alternate route: Let the Cognitive Services handle it

In [0]:
from synapse.ml.cognitive import TextSentiment
from synapse.ml.core.platform import find_secret

model = TextSentiment(
    textCol="text",
    outputCol="sentiment",
    subscriptionKey=find_secret("cognitive-api-key"),
).setLocation("eastus")

display(model.transform(test).limit(1))

rating text TextSentiment_945cae8860cd_error sentiment 1 A Review from the author of WATCH OUT The literature of September 11 is never attacked. When a book speaks of September 11 (or of terrorism in general), it is more or less guaranteed immunity from criticism; it will almost inescapably be greeted with sympathy. Jonathan Safran Foer's EXTREMELY LOUD AND INCREDIBLY CLOSE banks on such sympathy, on such reverence. The narrative, which concerns a nine-year-old boy named Oskar Schell in search of a key that would unlock / unshell the enigma of his dead father (a narrative stolen, in its basic outline, from Guenther Grass's DIE BLECHTROMMEL), could have been written entirely without its scattered references to the terrorist interventions. Nor is this trauma the only one presented in the novel: the others include Hiroshima / Nagasaki, the Staten Island Ferry crash, and the Dresden bombings. Each disaster is generalized to the point at which what is addressed is not a traumatizing event in its specificity, but historical trauma" itself and the overcoming of trauma through bereavement-inspired creation. Oskar, the insufferable brat, attempts to complete the work of mourning for his father, Thomas Schell, Jr., a victim of September 11, by compiling an almanac of self-inflicted wounds, the collage of images and letters which is the book we are "reading"---an almanac which, most likely, is written sometime in the indefinite future (Thomas Shell, Sr.'s manuscript of 4/12/78 is heavily edited (pp. 208-216). Who has done the editing? Almost certainly an older version of his grandson Oskar). If the term, "reading" even applies. Whenever a "pregnant" image is described, Foer literally re-presents it in the form of a pictorial image. When a flock of birds rises into the sky, it is not enough that we read of these birds---we must see them as well. Words may not be left in their invisibility; we are presented with supplementary photographs, illustrations, since mere verbality is not enough. (Indeed, the entire novel oozes with misologos --- the mistrust or hatred of language --- in terms of both its content and its form.) Photographs, yes, and also a superabundance of blank pages and nearly blank pages. Space is not used in the manner it is in the works of Edmond Jabes, for instance. Typography does not substitute for a well-wrought sentence. Foer abrogates himself of all responsibilities --- most specifically, the responsibility to write well. Why bother when the pyrotechnics of typography are at his disposal? As far as the writing is concerned, it is composed of nothing other than mind-numbingly, soul-deadeningly repetitive phrases ("heavy boots," "raison d'etre," etc.) and Sunday school platitudes: "Sometimes one simply wants to disappear" (184); "There's nothing wrong with not understanding yourself" (184); "Everything that's born has to die, which means our lives are like skyscrapers" (245); "How can you say I love you to someone you love?... It's always necessary" (314). Whenever the author writes something that he finds "beautiful" and "true" (165), he congratulates himself on his brilliance and tells us that that thing is "beautiful" and "true." The entire book reeks of such unearned profundity. We also learn that most dust is made up of human detritus---a very deep truth indeed, one that Foer also communicates in his essay, "Emptiness" (originally published in PLAYBOY) with all of the sanctimoniousness and self-righteousness of the faux naif who serves as the center of the novel, a Sunday school lecture in which we learn that famous musicians (Ringo Star) and scientists (Stephen Hawking) are unthreateningly approachable: everything is familiarized. Perhaps it is wrong to criticize Foer for including so many blank pages in EXTREMELY LOUD AND INCREDIBLY CLOSE, since the entire book is a vacuum: null space into which readers may project their own meanings. Joseph Suglia, Ph.D., the author of WATCH OUT" null List(null, List(0, mixed, nu